In [1]:
import pandas as pd
import numpy as np
import joblib
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

import os

print(os.getcwd())
print(os.getcwd())

import jquantsapi
import shap


C:\Users\trow126\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\trow126\Documents\J-Quants
C:\Users\trow126\Documents\J-Quants


In [2]:

# 指定されたパスが存在しない場合にのみディレクトリを作成する
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

filepath_stock_price = os.path.join(dir_path, 'stock_price_load.gz')
filepath_stock_list   = os.path.join(dir_path, 'stock_list_load.gz')


In [3]:
# J-Quants API から取得するデータの期間。ライトプランを想定し，5年間のデータ取得としています。
# 無料プランの場合は2年間（２週間ラグあり）取得できます。
HISTORICAL_DATA_YEARS = 10

start_dt = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=(HISTORICAL_DATA_YEARS*365))
end_dt = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
start_dt_yyyymmdd = start_dt.strftime("%Y%m%d")
end_dt_yyyymmdd = end_dt.strftime("%Y%m%d")

print( f'{start_dt_yyyymmdd}から{end_dt_yyyymmdd}までの記録を使用します。' )

20131221から20231219までの記録を使用します。


In [4]:
# J-Quants APIクライアントの初期化
cli = jquantsapi.Client(refresh_token=refresh_token)

if os.path.exists(filepath_stock_list):
   # ファイルが存在する場合、読み込む
   stock_list_load = pd.read_csv(filepath_stock_list, compression='gzip')
else:
   # ファイルが存在しない場合、新たに作成する
   stock_list_load = cli.get_listed_info()
   stock_list_load.to_csv(filepath_stock_list, compression='gzip', index=False)


# 株価情報のobject型をdatetime64[ns]型に変換
stock_list_load["Date"] = pd.to_datetime(stock_list_load["Date"])

# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_list_load["Code"] = stock_list_load["Code"].astype(str)
stock_list_load.loc[(stock_list_load["Code"].str.len() == 5) & (stock_list_load["Code"].str[-1] == "0"), "Code"] = stock_list_load.loc[(stock_list_load["Code"].str.len() == 5) & (stock_list_load["Code"].str[-1] == "0"), "Code"].str[:-1]
stock_list_load["Code"] = stock_list_load["Code"].astype(int)

# TOPIX500のtickerのみを取得する
categories = ['TOPIX Mid400', 'TOPIX Large70', 'TOPIX Core30']
tickers = stock_list_load[stock_list_load['ScaleCategory'].isin(categories)]['Code'].unique()
tickers = tickers.astype(str)

In [5]:
if os.path.exists(filepath_stock_price):
   # ファイルが存在する場合、読み込む
   stock_price_load = pd.read_csv(filepath_stock_price, compression='gzip')
else:
   # ファイルが存在しない場合、新たに作成する。初回は10分以上かかると思います。
   stock_price_load = cli.get_price_range(start_dt=start_dt, end_dt=end_dt)
   stock_price_load.to_csv(filepath_stock_price, compression='gzip', index=False)

# "Code"列を文字列型に変換
stock_price_load["Code"] = stock_price_load["Code"].astype(str)
# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_price_load.loc[(stock_price_load["Code"].str.len() == 5) & (stock_price_load["Code"].str[-1] == "0"), "Code"] = stock_price_load.loc[(stock_price_load["Code"].str.len() == 5) & (stock_price_load["Code"].str[-1] == "0"), "Code"].str[:-1]

stock_price_load

,Date,Code,Open,High,Low,Close,UpperLimit,LowerLimit,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
0,2013-12-20,1301,261.0,262.0,260.0,260.0,0,0,556000.0,145021000.0,1.0,2610.0,2620.0,2600.0,2600.0,55600.0
1,2013-12-24,1301,260.0,261.0,258.0,260.0,0,0,650000.0,168585000.0,1.0,2600.0,2610.0,2580.0,2600.0,65000.0
2,2013-12-25,1301,259.0,260.0,258.0,260.0,0,0,501000.0,129704000.0,1.0,2590.0,2600.0,2580.0,2600.0,50100.0
3,2013-12-26,1301,261.0,265.0,261.0,264.0,0,0,527000.0,138628000.0,1.0,2610.0,2650.0,2610.0,2640.0,52700.0
4,2013-12-27,1301,266.0,266.0,264.0,265.0,0,0,310000.0,82188000.0,1.0,2660.0,2660.0,2640.0,2650.0,31000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9672863,2023-12-12,9997,615.0,617.0,608.0,608.0,0,0,338400.0,206919200.0,1.0,615.0,617.0,608.0,608.0,338400.0
9672864,2023-12-13,9997,609.0,610.0,600.0,601.0,0,0,469500.0,283295100.0,1.0,609.0,610.0,600.0,601.0,469500.0
9672865,2023-12-14,9997,601.0,603.0,591.0,596.0,0,0,649900.0,387994900.0,1.0,601.0,603.0,591.0,596.0,649900.0
9672866,2023-12-15,9997,595.0,597.0,590.0,597.0,0,0,625100.0,371592500.0,1.0,595.0,597.0,590.0,597.0,625100.0


In [6]:
df_ohlcv = stock_price_load[stock_price_load['Code'].isin(tickers)]
df_ohlcv

,Date,Code,Open,High,Low,Close,UpperLimit,LowerLimit,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
51176,2013-12-20,1332,234.0,237.0,234.0,235.0,0,0,2116500.0,4.972783e+08,1.0,234.0,237.0,234.0,235.0,2116500.0
51177,2013-12-24,1332,235.0,235.0,232.0,232.0,0,0,1788000.0,4.180801e+08,1.0,235.0,235.0,232.0,232.0,1788000.0
51178,2013-12-25,1332,232.0,236.0,232.0,234.0,0,0,2199000.0,5.141982e+08,1.0,232.0,236.0,232.0,234.0,2199000.0
51179,2013-12-26,1332,233.0,238.0,233.0,236.0,0,0,1807800.0,4.264282e+08,1.0,233.0,238.0,233.0,236.0,1807800.0
51180,2013-12-27,1332,238.0,243.0,237.0,242.0,0,0,2652300.0,6.390824e+08,1.0,238.0,243.0,237.0,242.0,2652300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9654387,2023-12-12,9989,4562.0,4585.0,4538.0,4563.0,0,0,216400.0,9.868495e+08,1.0,4562.0,4585.0,4538.0,4563.0,216400.0
9654388,2023-12-13,9989,4604.0,4604.0,4492.0,4503.0,0,0,188400.0,8.508363e+08,1.0,4604.0,4604.0,4492.0,4503.0,188400.0
9654389,2023-12-14,9989,4474.0,4509.0,4445.0,4492.0,0,0,280700.0,1.258709e+09,1.0,4474.0,4509.0,4445.0,4492.0,280700.0
9654390,2023-12-15,9989,4475.0,4479.0,4365.0,4386.0,0,0,327300.0,1.440833e+09,1.0,4475.0,4479.0,4365.0,4386.0,327300.0


In [7]:
def calc_features(df_ohlcv):
    # Creating a new DataFrame to store the features
    df_feats = df_ohlcv[['Date','Code','Close']]

    df_feats['ror1']  = df_ohlcv['Close'].pct_change(1).fillna(0)
    df_feats['ror5']  = df_ohlcv['Close'].pct_change(4).shift(1).fillna(0)
    df_feats['ror25'] = df_ohlcv['Close'].pct_change(20).shift(5).fillna(0)
    df_feats['MA5_gap']  = (df_ohlcv['Close'].rolling(5).mean() / df_ohlcv['Close'] - 1.0).fillna(0)
    df_feats['MA25_gap']  = (df_ohlcv['Close'].rolling(25).mean() / df_ohlcv['Close'] - 1.0).fillna(0)
    df_feats['volatility5']  = np.log(df_ohlcv['Close']).diff().rolling(5).std().fillna(0)
    df_feats['volatility25']  = np.log(df_ohlcv['Close']).diff().rolling(25).std().fillna(0)
    df_feats['ror1_lag1']  = df_feats['ror1'].shift().fillna(0)

    df_feats['target'] = (df_ohlcv['Close'] / df_ohlcv['Open'] - 1.0).shift(-1)

    return df_feats

tqdm.pandas()

# 銘柄ごとに特徴量の計算
df_feats = df_ohlcv.groupby('Code').progress_apply(calc_features)

df_feats

  0%|          | 0/498 [00:00<?, ?it/s]

100%|██████████| 498/498 [00:01<00:00, 295.94it/s]


,Date,Code,Close,ror1,ror5,ror25,MA5_gap,MA25_gap,volatility5,volatility25,ror1_lag1,target
51176,2013-12-20,1332,235.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.012766
51177,2013-12-24,1332,232.0,-0.012766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008621
51178,2013-12-25,1332,234.0,0.008621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.012766,0.012876
51179,2013-12-26,1332,236.0,0.008547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008621,0.016807
51180,2013-12-27,1332,242.0,0.025424,0.000000,0.000000,-0.025620,0.000000,0.000000,0.000000,0.008547,-0.016461
...,...,...,...,...,...,...,...,...,...,...,...,...
9654387,2023-12-12,9989,4563.0,-0.000219,-0.019759,0.158497,0.010256,-0.044716,0.019088,0.013857,0.008842,-0.021937
9654388,2023-12-13,9989,4503.0,-0.013149,-0.030181,0.192649,0.014746,-0.027031,0.017459,0.013513,-0.000219,0.004023
9654389,2023-12-14,9989,4492.0,-0.002443,-0.040486,0.181223,0.008281,-0.020027,0.017468,0.013595,-0.013149,-0.019888
9654390,2023-12-15,9989,4386.0,-0.023598,-0.007073,0.134688,0.026357,0.007296,0.012620,0.014766,-0.002443,-0.003464


In [8]:
from ta import add_all_ta_features

add_all_ta_features_path = os.path.join(dir_path, 'add_all_ta_features.gz')

def create_add_all_ta_features(df_ohlcv):
    df_feats = add_all_ta_features(df_ohlcv, open="Open", high="High", low="Low", close="Close", volume="Volume")
    # df_feats = df_feats.std().fillna(0)
    return df_feats

tqdm.pandas()

add_all_ta_features_load = df_ohlcv.groupby('Code').progress_apply(create_add_all_ta_features)
add_all_ta_features_load.to_csv(add_all_ta_features_path, compression='gzip', index=False)

add_all_ta_features_load

  0%|          | 0/498 [00:00<?, ?it/s]

100%|██████████| 498/498 [02:05<00:00,  3.98it/s]


,Date,Code,Open,High,Low,Close,UpperLimit,LowerLimit,Volume,TurnoverValue,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
51176,2013-12-20,1332,234.0,237.0,234.0,235.0,0,0,2116500.0,4.972783e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
51177,2013-12-24,1332,235.0,235.0,232.0,232.0,0,0,1788000.0,4.180801e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.276596,-1.284814,-1.276596
51178,2013-12-25,1332,232.0,236.0,232.0,234.0,0,0,2199000.0,5.141982e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.862069,0.858374,-0.425532
51179,2013-12-26,1332,233.0,238.0,233.0,236.0,0,0,1807800.0,4.264282e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.854701,0.851069,0.425532
51180,2013-12-27,1332,238.0,243.0,237.0,242.0,0,0,2652300.0,6.390824e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.542373,2.510592,2.978723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9654387,2023-12-12,9989,4562.0,4585.0,4538.0,4563.0,0,0,216400.0,9.868495e+08,...,2.772349,2.745041,0.027308,-3.352785,1.874483,-5.227268,NaN,-0.021911,-0.021913,-1.659483
9654388,2023-12-13,9989,4604.0,4604.0,4492.0,4503.0,0,0,188400.0,8.508363e+08,...,2.516875,2.699408,-0.182533,-7.162744,0.067037,-7.229781,NaN,-1.314924,-1.323646,-2.952586
9654389,2023-12-14,9989,4474.0,4509.0,4445.0,4492.0,0,0,280700.0,1.258709e+09,...,2.268687,2.613264,-0.344577,-8.110649,-1.568500,-6.542149,NaN,-0.244282,-0.244580,-3.189655
9654390,2023-12-15,9989,4475.0,4479.0,4365.0,4386.0,0,0,327300.0,1.440833e+09,...,1.860677,2.462746,-0.602070,-7.688777,-2.792555,-4.896221,NaN,-2.359751,-2.388039,-5.474138


In [9]:
df_feats = df_feats.join(add_all_ta_features_load.iloc[:, 16:])
df_feats = df_feats.replace(np.nan, 0)


In [10]:
df_feats

,Date,Code,Close,ror1,ror5,ror25,MA5_gap,MA25_gap,volatility5,volatility25,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
51176,2013-12-20,1332,235.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
51177,2013-12-24,1332,232.0,-0.012766,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,-1.276596,-1.284814,-1.276596
51178,2013-12-25,1332,234.0,0.008621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.862069,0.858374,-0.425532
51179,2013-12-26,1332,236.0,0.008547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.854701,0.851069,0.425532
51180,2013-12-27,1332,242.0,0.025424,0.000000,0.000000,-0.025620,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.542373,2.510592,2.978723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9654387,2023-12-12,9989,4563.0,-0.000219,-0.019759,0.158497,0.010256,-0.044716,0.019088,0.013857,...,2.772349,2.745041,0.027308,-3.352785,1.874483,-5.227268,0.0,-0.021911,-0.021913,-1.659483
9654388,2023-12-13,9989,4503.0,-0.013149,-0.030181,0.192649,0.014746,-0.027031,0.017459,0.013513,...,2.516875,2.699408,-0.182533,-7.162744,0.067037,-7.229781,0.0,-1.314924,-1.323646,-2.952586
9654389,2023-12-14,9989,4492.0,-0.002443,-0.040486,0.181223,0.008281,-0.020027,0.017468,0.013595,...,2.268687,2.613264,-0.344577,-8.110649,-1.568500,-6.542149,0.0,-0.244282,-0.244580,-3.189655
9654390,2023-12-15,9989,4386.0,-0.023598,-0.007073,0.134688,0.026357,0.007296,0.012620,0.014766,...,1.860677,2.462746,-0.602070,-7.688777,-2.792555,-4.896221,0.0,-2.359751,-2.388039,-5.474138


In [11]:
display(df_feats.iloc[:,3:].columns)

Index(['ror1', 'ror5', 'ror25', 'MA5_gap', 'MA25_gap', 'volatility5',
       'volatility25', 'ror1_lag1', 'target', 'volume_adi', 'volume_obv',
       'volume_cmf', 'volume_fi', 'volume_em', 'volume_sma_em', 'volume_vpt',
       'volume_vwap', 'volume_mfi', 'volume_nvi', 'volatility_bbm',
       'volatility_bbh', 'volatility_bbl', 'volatility_bbw', 'volatility_bbp',
       'volatility_bbhi', 'volatility_bbli', 'volatility_kcc',
       'volatility_kch', 'volatility_kcl', 'volatility_kcw', 'volatility_kcp',
       'volatility_kchi', 'volatility_kcli', 'volatility_dcl',
       'volatility_dch', 'volatility_dcm', 'volatility_dcw', 'volatility_dcp',
       'volatility_atr', 'volatility_ui', 'trend_macd', 'trend_macd_signal',
       'trend_macd_diff', 'trend_sma_fast', 'trend_sma_slow', 'trend_ema_fast',
       'trend_ema_slow', 'trend_vortex_ind_pos', 'trend_vortex_ind_neg',
       'trend_vortex_ind_diff', 'trend_trix', 'trend_mass_index', 'trend_dpo',
       'trend_kst', 'trend_kst_sig', '

In [12]:
# 五分位化する特徴量
# features = ['ror1', 'ror5', 'ror25', 'MA5_gap', 'MA25_gap', 'volatility5', 'volatility25', 'ror1_lag1']
features = df_feats.iloc[:,3:].columns

for feature in features:
    df_feats[feature] = pd.qcut(df_feats[feature], 5, labels=False) - 2.0

df_feats

ValueError: Bin edges must be unique: array([0., 0., 0., 0., 0., 1.]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [ ]:
TRAIN_DATA_YEARS = 7
TEST_DATA_YEARS = 3
# TRAIN_DATA_YEARS + TEST_DATA_YEARS <= HISTORICAL_DATA_YEARS となるようにしてください

train_start_dt = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) -  timedelta(days=(HISTORICAL_DATA_YEARS*365))
train_end_dt = train_start_dt + timedelta(days=(TRAIN_DATA_YEARS*365))
test_start_dt = train_end_dt + timedelta(days=1)
test_end_dt = test_start_dt + timedelta(days=(TEST_DATA_YEARS*365))

train_start_dt_yyyymmdd = train_start_dt.strftime("%Y-%m-%d")
train_end_dt_yyyymmdd = train_end_dt.strftime("%Y-%m-%d")
test_start_dt_yyyymmdd = test_start_dt.strftime("%Y-%m-%d")
test_end_dt_yyyymmdd = test_end_dt.strftime("%Y-%m-%d")

print( f'訓練データ　: {train_start_dt_yyyymmdd}から{train_end_dt_yyyymmdd}までの記録を使用します。' )
print( f'テストデータ: {test_start_dt_yyyymmdd}から{test_end_dt_yyyymmdd}までの記録を使用します。' )

In [ ]:
# 特徴量として使用するカラム
features = [
    'ror1',
    'ror5',
    'ror25',
    'MA5_gap',
    'MA25_gap',
    'volatility5',
    'volatility25',
    'ror1_lag1',
    ]

# ターゲット変数として使用するカラム
target = 'target'

# 2018年から2021年の3年間をデータを訓練データ、2022年から2024年の2年間のデータをテストデータにします。
df_train = df_feats[(df_feats['Date'] >= '2018-01-01') & (df_feats['Date'] < '2022-01-01')]
df_test = df_feats[(df_feats['Date'] >= '2022-01-01') & (df_feats['Date'] < '2024-01-01')]

# 学習データとテストデータの特徴量とターゲットを定義
X_train = df_train[features].fillna(0)
y_train = df_train[target].fillna(0)

X_test = df_test[features].fillna(0)
y_test = df_test[target].fillna(0)

In [ ]:
# LightGBMモデルの学習
model = lgb.LGBMRegressor(random_state=42, verbose=-1, max_depth=4)
model.fit(X_train, y_train)

# 訓練データでの予測値
df_train['y_pred'] = model.predict(X_train)
# テストデータでの予測値
df_test['y_pred'] = model.predict(X_test)


In [ ]:
# 訓練データ，テストデータのスコアを計測。テストデータに対し，訓練データが過学習していないか確認する。(例：訓練データの正答率が70％越え等は非現実的）

def score(df, target_name='target', pred_name='y_pred'):
    '''Takes df and calculates spearm correlation from pre-defined cols'''
    # method="first" breaks ties based on order in array
    return np.corrcoef(
        df[target_name].fillna(0),
        df[pred_name].rank(pct=True, method="first")
    )[0,1]

def run_analytics(era_scores):
    print(f"Mean Correlation: {era_scores.mean():.4f}")
    print(f"Median Correlation: {era_scores.median():.4f}")
    print(f"Standard Deviation: {era_scores.std():.4f}")
    print('\n')
    print(f"Mean Pseudo-Sharpe: {era_scores.mean()/era_scores.std():.4f}")
    print(f"Median Pseudo-Sharpe: {era_scores.median()/era_scores.std():.4f}")
    print('\n')
    print(f'Hit Rate (% positive eras): {era_scores.apply(lambda x: np.sign(x)).value_counts()[1]/len(era_scores):.2%}')

    era_scores.rolling(10).mean().plot(kind='line', title='Rolling Per Era Correlation Mean', figsize=(15,4))
    plt.axhline(y=0.0, color="r", linestyle="--"); plt.show()

    era_scores.cumsum().plot(title='Cumulative Sum of Era Scores', figsize=(15,4))
    plt.axhline(y=0.0, color="r", linestyle="--"); plt.show()



print('--- Train Score ---')
train_scores = df_train.groupby('Date').apply(score)
run_analytics(train_scores)
print('\n')

print('--- Test Score ---')
test_scores = df_test.groupby('Date').apply(score)
run_analytics(test_scores)
print('\n')

In [ ]:
# SHAPの計算
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

# 特徴量の重要度の可視化
shap.summary_plot(shap_values, X_test)